# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [46]:
// TODO: Replace <FILL IN> with appropriate code

val housing = spark.read.format("csv").option("header", "true").option("inferSchema","true")load("data/housing.csv")

housing = [longitude: double, latitude: double ... 8 more fields]


[longitude: double, latitude: double ... 8 more fields]

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [47]:
// TODO: Replace <FILL IN> with appropriate code

housing.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Print the number of records in the dataset.

In [48]:
// TODO: Replace <FILL IN> with appropriate code

housing.count()

20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [49]:
// TODO: Replace <FILL IN> with appropriate code

housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Print the number of records with population more than 10000.

In [50]:
// TODO: Replace <FILL IN> with appropriate code

housing.where("population > 10000").count()

23

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [51]:
// TODO: Replace <FILL IN> with appropriate code

val a=housing.select("housing_median_age","total_rooms","median_house_value","population").describe()
a.show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



a = [summary: string, housing_median_age: string ... 3 more fields]


[summary: string, housing_median_age: string ... 3 more fields]

Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [52]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.sql.functions._

housing.select(max("housing_median_age"),min("total_rooms"),avg("median_house_value")).show()

+-----------------------+----------------+-----------------------+
|max(housing_median_age)|min(total_rooms)|avg(median_house_value)|
+-----------------------+----------------+-----------------------+
|                   52.0|             2.0|     206855.81690891474|
+-----------------------+----------------+-----------------------+



## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [53]:
// TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity").agg(count("ocean_proximity")).sort(desc("ocean_proximity")).show()

+---------------+----------------------+
|ocean_proximity|count(ocean_proximity)|
+---------------+----------------------+
|     NEAR OCEAN|                  2658|
|       NEAR BAY|                  2290|
|         ISLAND|                     5|
|         INLAND|                  6551|
|      <1H OCEAN|                  9136|
+---------------+----------------------+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [54]:
// TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity").agg(avg("median_house_value") as "avg_vale").sort(desc("ocean_proximity")).show()

+---------------+------------------+
|ocean_proximity|          avg_vale|
+---------------+------------------+
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|         ISLAND|          380440.0|
|         INLAND|124805.39200122119|
|      <1H OCEAN|240084.28546409807|
+---------------+------------------+



Rewrite the above question in SQL.

In [55]:
// TODO: Replace <FILL IN> with appropriate code

housing.createOrReplaceTempView("df")
spark.sql("select ocean_proximity,avg(median_house_value) as avg_value from df group by ocean_proximity order by ocean_proximity desc ").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|         ISLAND|          380440.0|
|         INLAND|124805.39200122119|
|      <1H OCEAN|240084.28546409807|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [56]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("housing_median_age","total_rooms","median_house_value")).setOutputCol("features")

val housingAttrs = va.transform(housing)

housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|[41.0,880.0,45260...|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|[21.0,7099.0,3585...|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|[52.0,1467.0,3521...|
|  -122.25|   37.85|              52.0|     12

va = vecAssembler_aa89b12b8bfd
housingAttrs = [longitude: double, latitude: double ... 9 more fields]


[longitude: double, latitude: double ... 9 more fields]

In [57]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs,"features").head()

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0                   -0.36126220122231784  0.10562341249318154  
-0.36126220122231784  1.0                   0.13415311380654338  
0.10562341249318154   0.13415311380654338   1.0                  


coeff = 


1.0                   -0.36126220122231784  0.10562341249318154
-0.36126220122231784  1.0                   0.13415311380654338
0.10562341249318154   0.13415311380654338   1.0


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

rooms_per_household = total_rooms / households;
bedrooms_per_room = total_bedrooms / total_rooms;
population_per_household = population / households;

In [58]:
// TODO: Replace <FILL IN> with appropriate code

val housingCol1 = housing.withColumn("rooms_per_household",expr("total_rooms/households"))
val housingCol2 = housingCol1.withColumn("bedrooms_per_room",expr("total_bedrooms/total_rooms"))
val housingExtra = housingCol2.withColumn("population_per_household",expr("population/households"))

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



housingCol1 = [longitude: double, latitude: double ... 9 more fields]
housingCol2 = [longitude: double, latitude: double ... 10 more fields]
housingExtra = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [59]:
// TODO: Replace <FILL IN> with appropriate code

val renamedHousing = housingExtra.withColumnRenamed("median_house_value","label");

renamedHousing = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [60]:
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel = label
colCat = ocean_proximity
colNum = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)


Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)

## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [61]:
// TODO: Replace <FILL IN> with appropriate code

for (c <- colNum) {
    println(c)
   val a =renamedHousing.where(renamedHousing.col(c).isNull).count()

   println(a) 
}

longitude
0
latitude
0
housing_median_age
0
total_rooms
0
total_bedrooms
207
population
0
households
0
median_income
0
rooms_per_household
0
bedrooms_per_room
207
population_per_household
0


As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [62]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median").setInputCols(Array("total_bedrooms","bedrooms_per_room")).setOutputCols(Array("total_bedrooms","bedrooms_per_room"))                                    
val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows



imputer = imputer_e02ad21df895
imputedHousing = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [63]:
imputedHousing.columns

Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, label, ocean_proximity, rooms_per_household, bedrooms_per_room, population_per_household)

In [64]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("featuresFlat")
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler().setInputCol("featuresFlat").setOutputCol("scalar")
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|        featuresFlat|              scalar|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069...|
|  -122.22|   37.86|              21.0|     7099.0|        1

va = vecAssembler_e7d6255ac118
featuredHousing = [longitude: double, latitude: double ... 12 more fields]
scaler = stdScal_7d188b486ae1
scaledHousing = [longitude: double, latitude: double ... 13 more fields]


[longitude: double, latitude: double ... 13 more fields]

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [65]:
// TODO: Replace <FILL IN> with appropriate code

renamedHousing.select("ocean_proximity").distinct().count()

5

### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [66]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol(colCat).setOutputCol("catInd")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|catInd|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|   3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.238137082601054|0.15579659106916466|       2.109841827768014|   3.0|


indexer = strIdx_b22c59dad69a
idxHousing = [longitude: double, latitude: double ... 12 more fields]


[longitude: double, latitude: double ... 12 more fields]

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [67]:
indexer.fit(renamedHousing).labels

Array(<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND)

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [68]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.OneHotEncoderEstimator



val encoder = new OneHotEncoderEstimator().setInputCols(Array("catInd")).setOutputCols(Array("catEncode"))
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+------+-------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|catInd|    catEncode|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+------+-------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|   3.0|(4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.2381370826010

encoder = oneHotEncoder_3bb9c3424f7c
ohHousing = [longitude: double, latitude: double ... 13 more fields]


[longitude: double, latitude: double ... 13 more fields]

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [69]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(imputer, va,scaler))
val catPipeline = new Pipeline().setStages(Array(indexer, encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))
val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+------+-------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|        featuresFlat|              scalar|catInd|    catEncode|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+------+-------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069..

numPipeline = pipeline_990d70c9b07d
catPipeline = pipeline_32d7eb4550a0
pipeline = pipeline_f7a9ba0c6c0d
newHousing = [longitude: double, latitude: double ... 15 more fields]


[longitude: double, latitude: double ... 15 more fields]

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [70]:
// TODO: Replace <FILL IN> with appropriate code

//val cols = newHousing.columns.filter(_ != colCat)
val cols=Array("scalar","catEncode")
println(cols);
val va2 = new VectorAssembler().setInputCols(cols).setOutputCol("features")
val dataset = va2.transform(newHousing).select("features", "label")

dataset.show(5)

[Ljava.lang.String;@2aea4b2e
+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-61.007269596069...|452600.0|
|[-61.002278409814...|358500.0|
|[-61.012260782324...|352100.0|
|[-61.017251968579...|341300.0|
|[-61.017251968579...|342200.0|
+--------------------+--------+
only showing top 5 rows



cols = Array(scalar, catEncode)
va2 = vecAssembler_4f39505f1e29
dataset = [features: vector, label: double]


[features: vector, label: double]

---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [71]:
// TODO: Replace <FILL IN> with appropriate code
val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

testSet.show()

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-62.040445150874...|103600.0|
|[-61.980550915813...|107000.0|
|[-61.980550915813...| 67000.0|
|[-61.975559729558...| 81300.0|
|[-61.975559729558...| 62500.0|
|[-61.975559729558...|109400.0|
|[-61.970568543303...| 85400.0|
|[-61.970568543303...| 75500.0|
|[-61.970568543303...| 69500.0|
|[-61.970568543303...| 75000.0|
|[-61.955594984538...| 86900.0|
|[-61.945612612028...| 96100.0|
|[-61.945612612028...| 81700.0|
|[-61.940621425772...| 92600.0|
|[-61.935630239517...| 87500.0|
|[-61.935630239517...|105800.0|
|[-61.935630239517...| 99600.0|
|[-61.930639053262...|150000.0|
|[-61.895700749477...| 75600.0|
|[-61.810850583140...| 74000.0|
+--------------------+--------+
only showing top 20 rows



trainSet = [features: vector, label: double]
testSet = [features: vector, label: double]


[features: vector, label: double]

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [72]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression().setMaxIter(10)
.setFeaturesCol("features").setLabelCol("label").setSolver("normal")
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: ${lrModel.coefficients}, Intercept: ${lrModel.intercept}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")

Coefficients: [-54080.65702468992,-54786.98107947752,13704.419413109707,7765.243730495444,5884.992234270932,-45034.45920756295,36589.782478416775,78837.69261220499,4865.093837480598,15809.37739159288,311.40257418509253,-166783.74064588608,-201656.30542965676,-162230.18420913685,-170657.03635468753], Intercept: -2198138.5121909687
RMSE: 67897.7352489894


lr = linReg_2d1b3c124310
lrModel = linReg_2d1b3c124310
trainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@4cf05f4e


org.apache.spark.ml.regression.LinearRegressionTrainingSummary@4cf05f4e

Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [73]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setMetricName("rmse")
.setPredictionCol("prediction")
.setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|143053.50076636067|103600.0|[-62.040445150874...|
|180206.25448934734|107000.0|[-61.980550915813...|
| 148208.4421245549| 67000.0|[-61.980550915813...|
| 140717.0423596762| 81300.0|[-61.975559729558...|
|159266.70354691474| 62500.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67973.60239352743


predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_a535f682f68b
rmse = 67973.60239352743


67973.60239352743

## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [74]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setLabelCol("label").setFeaturesCol("features")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setMetricName("rmse")

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|175278.03280609255|103600.0|[-62.040445150874...|
|175278.03280609255|107000.0|[-61.980550915813...|
| 144759.2406311637| 67000.0|[-61.980550915813...|
| 144759.2406311637| 81300.0|[-61.975559729558...|
|151033.56890459365| 62500.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67963.45058575872


dt = dtr_8169d0a8ba8c
dtModel = DecisionTreeRegressionModel (uid=dtr_8169d0a8ba8c) of depth 5 with 63 nodes
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_5a576dc55273
rmse = 67963.45058575872


67963.45058575872

## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [75]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().<FILL IN>

// train the model
val rfModel = rf.<FILL IN>

// make predictions on the test data
val predictions = rfModel.<FILL IN>
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().<FILL IN>
val rmse = evaluator.<FILL IN>
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

Name: Compile Error
Message: <console>:64: error: value < is not a member of org.apache.spark.ml.regression.RandomForestRegressor
       val rf = new RandomForestRegressor().<FILL IN>
                                            ^
<console>:64: error: not found: value IN
       val rf = new RandomForestRegressor().<FILL IN>
                                                  ^
<console>:74: error: value < is not a member of org.apache.spark.ml.evaluation.RegressionEvaluator
       val evaluator = new RegressionEvaluator().<FILL IN>
                                                 ^
<console>:74: error: not found: value IN
       val evaluator = new RegressionEvaluator().<FILL IN>
                                                       ^

StackTrace: 

## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [76]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().<FILL IN>

// train the model
val gbModel = gb.<FILL IN>

// make predictions on the test data
val predictions = gbModel.<FILL IN>
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().<FILL IN>
val rmse = evaluator.<FILL IN>
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

Name: Compile Error
Message: <console>:64: error: value < is not a member of org.apache.spark.ml.regression.GBTRegressor
       val gb = new GBTRegressor().<FILL IN>
                                   ^
<console>:64: error: not found: value IN
       val gb = new GBTRegressor().<FILL IN>
                                         ^
<console>:74: error: value < is not a member of org.apache.spark.ml.evaluation.RegressionEvaluator
       val evaluator = new RegressionEvaluator().<FILL IN>
                                                 ^
<console>:74: error: not found: value IN
       val evaluator = new RegressionEvaluator().<FILL IN>
                                                       ^

StackTrace: 

---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [77]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder().<FILL IN>

val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName(<FILL IN>)
val cv = new CrossValidator().<FILL IN>
val cvModel = cv.<FILL IN>

val predictions = cvModel.<FILL IN>
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.<FILL IN>
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

Name: Compile Error
Message: <console>:9: error: in XML literal: '=' expected instead of '>'
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName(<FILL IN>)
                                                                                                                    ^
<console>:9: error: in XML literal: ' or " delimited attribute value or '{' scala-expr '}' expected
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName(<FILL IN>)
                                                                                                                     ^
<console>:10: error: in XML literal: '=' expected instead of 'c'
val cv = new CrossValidator().<FILL IN>
    ^
<console>:10: error: in XML literal: ' or " delimited attribute value or '{' scala-expr '}' expected
val cv = new CrossValidator().<FILL IN>
     ^
<console>:10: error: in XML literal: '>' expected instead of '='
val cv =

---
# 7. Custom transformer
At the end of part two, we added extra columns to the `housing` dataset. Here, we are going to implement a Transformer to do the same task. The Transformer should take the name of two input columns `inputCol1` and `inputCol2`, as well as the name of ouput column `outputCol`. It, then, computes `inputCol1` divided by `inputCol2`, and adds its result as a new column to the dataset. The details of the implemeting a custom Tranfomer is explained [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types). Please read it before before starting to implement it.

First, define the given parameters of the Transformer and implement a method to validate their schemas (`StructType`).

In [78]:
import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}

trait MyParams extends Params {
    final val inputCol1 = <FILL IN>
    final val inputCol2 = <FILL IN>
    final val outputCol = <FILL IN>
    
  protected def validateAndTransformSchema(schema: StructType): StructType = {
    <FILL IN>  
  }
}


Name: Compile Error
Message: <console>:5: error: in XML literal: '=' expected instead of '>'
    final val inputCol1 = <FILL IN>
                                  ^
<console>:6: error: in XML literal: ' or " delimited attribute value or '{' scala-expr '}' expected
    final val inputCol2 = <FILL IN>
    ^
<console>:6: error: in XML literal: whitespace expected
    final val inputCol2 = <FILL IN>
     ^
<console>:6: error: in XML literal: '=' expected instead of 'v'
    final val inputCol2 = <FILL IN>
          ^
<console>:6: error: in XML literal: ' or " delimited attribute value or '{' scala-expr '}' expected
    final val inputCol2 = <FILL IN>
           ^
<console>:6: error: in XML literal: whitespace expected
    final val inputCol2 = <FILL IN>
            ^
<console>:6: error: in XML literal: '>' expected instead of ' '
    final val inputCol2 = <FILL IN>
             ^
<console>:6: error: in XML literal: '=' expected instead of '>'
    final val inputCol2 = <FILL IN>
            

Then, extend the class `Transformer`, and implement its setter functions for the input and output columns, and call then `setInputCol1`, `setInputCol2`, and `setOutputCol`. Morever, you need to override the methods `copy`, `transformSchema`, and the `transform`. The details of what you need to cover in these methods is given [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types).

In [79]:
import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions.{col, udf}

class MyTransformer(override val uid: String) extends Transformer with MyParams {
    def this() = this(Identifiable.randomUID("configurablewordcount"))
    
    def setInputCol1(value: String): <FILL IN>
    
    def setInputCol2(value: String): <FILL IN>
    
    def setOutputCol(value: String): <FILL IN>

    override def copy(extra: ParamMap) <FILL IN>

    override def transformSchema(schema: StructType) <FILL IN>
    
    override def transform(dataset: Dataset[_]): DataFrame = {
        <FILL IN>
  }
}

Name: Compile Error
Message: <console>:11: error: identifier expected but $XMLSTART$< found.
    def setInputCol1(value: String): <FILL IN>
                                     ^
<console>:13: error: identifier expected but $XMLSTART$< found.
    def setInputCol2(value: String): <FILL IN>
                                     ^
<console>:15: error: identifier expected but $XMLSTART$< found.
    def setOutputCol(value: String): <FILL IN>
                                     ^
<console>:17: error: '=' expected but $XMLSTART$< found.
    override def copy(extra: ParamMap) <FILL IN>
                                       ^
<console>:19: error: illegal start of simple expression
    override def transformSchema(schema: StructType) <FILL IN>
^
<console>:19: error: '=' expected but $XMLSTART$< found.
    override def transformSchema(schema: StructType) <FILL IN>
                                                     ^
<console>:21: error: illegal start of simple expression
    override def trans

Now, an instance of `MyTransformer`, and set the input columns `total_rooms` and `households`, and the output column `rooms_per_household` and run it over the `housing` dataset.

In [80]:
val myTransformer = new MyTransformer().setInputCol1("total_rooms").setInputCol2("households").setOutputCol("rooms_per_household")

val myDataset = myTransformer.transform(housing).select("rooms_per_household").show(5)

Name: Compile Error
Message: <console>:57: error: not found: type MyTransformer
       val myTransformer = new MyTransformer().setInputCol1("total_rooms").setInputCol2("households").setOutputCol("rooms_per_household")
                               ^

StackTrace: 

---
# 8. Custom estimator (predictor)
Now, it's time to implement your own linear regression with gradient descent algorithm as a brand new Estimator. The whole code of the Estimator is given to you, and you do not need to implement anything. It is just a sample that shows how to build a custom Estimator.

The gradient descent update for linear regression is:
$$
w_{i+1} = w_{i} - \alpha_{i} \sum\limits_{j=1}^n (w_i^\top x_j - y_j)x_j
$$

where $i$ is the iteration number of the gradient descent algorithm, and $j$ identifies the observation. Here, $w$ represents an array of weights that is the same size as the array of features and provides a weight for each of the features when finally computing the label prediction in the form:

$$
prediction = w^\top \cdot\ x
$$

where $w$ is the final array of weights computed by the gradient descent, $x$ is the array of features of the observation point and $prediction$ is the label we predict should be associated to this observation.

The given `Helper` class implements the helper methods:
* `dot`: implements the dot product of two vectors and the dot product of a vector and a scalar
* `sum`: implements addition of two vectors
* `fill`: creates a vector of predefined size and initialize it with the predefined value

What you need to do is to implement the methods of the Linear Regresstion class `LR`, which are
* `rmsd`: computes the Root Mean Square Error of a given RDD of tuples of (label, prediction) using the formula:
$$
rmse = \sqrt{\frac{\sum\limits_{i=1}^n (label - prediction)^2}{n}}
$$
* `gradientSummand`: computes the following formula:
$$
gs_{ij} = (w_i^\top x_j - y_j)x_j
$$
* `gradient`: computes the following formula:
$$
gradient = \sum\limits_{j=1}^n gs_{ij}
$$

In [81]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

case class Instance(label: Double, features: Vector)

object Helper extends Serializable {
  def dot(v1: Vector, v2: Vector): Double = {
    val m = Matrices.dense(1, v1.size, v1.toArray)
    m.multiply(v2).values(0)
  }

  def dot(v: Vector, s: Double): Vector = {
    val baseArray = v.toArray.map(vi => vi * s)
    Vectors.dense(baseArray)
  }

  def sumVectors(v1: Vector, v2: Vector): Vector = {
    val baseArray = ((v1.toArray) zip (v2.toArray)).map { case (val1, val2) => val1 + val2 }
    Vectors.dense(baseArray)
  }

  def fillVector(size: Int, fillVal: Double): Vector = Vectors.dense(Array.fill[Double](size)(fillVal));
}

defined class Instance
defined object Helper


In [82]:
class LR() extends Serializable {
  def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
    val regressionMetrics = new RegressionMetrics(labelsAndPreds)
    regressionMetrics.rootMeanSquaredError
  }
  
  def gradientSummand(weights: Vector, lp: Instance): Vector = {
    val mult = (Helper.dot(weights, lp.features) - lp.label)
    val seq = (0 to lp.features.size - 1).map(i => lp.features(i) * mult)
    return Vectors.dense(seq.toArray)
  }
  
  def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
    val n = trainData.count()
    val d = trainData.take(1)(0).features.size
    var w = Helper.fillVector(d, 0)
    val alpha = 1.0
    val errorTrain = Array.fill[Double](numIters)(0.0)

    for (i <- 0 until numIters) {
      val labelsAndPredsTrain = trainData.map(lp => (lp.label, Helper.dot(w, lp.features)))
      errorTrain(i) = calcRMSE(labelsAndPredsTrain)

      val gradient = trainData.map(lp => gradientSummand(w, lp)).reduce((v1, v2) => Helper.sumVectors(v1, v2))
      val alpha_i = alpha / (n * scala.math.sqrt(i + 1))
      val wAux = Helper.dot(gradient, (-1) * alpha_i)
      w = Helper.sumVectors(w, wAux)
    }
    (w, errorTrain)
  }
}

Name: Compile Error
Message: <console>:18: error: not found: type RDD
         def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
                                      ^
<console>:19: error: not found: type RegressionMetrics
           val regressionMetrics = new RegressionMetrics(labelsAndPreds)
                                       ^
<console>:23: error: type Vector takes type parameters
         def gradientSummand(weights: Vector, lp: Instance): Vector = {
                                                             ^
<console>:23: error: type Vector takes type parameters
         def gradientSummand(weights: Vector, lp: Instance): Vector = {
                                      ^
<console>:26: error: not found: value Vectors
           return Vectors.dense(seq.toArray)
                  ^
<console>:29: error: type Vector takes type parameters
         def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
                    

In [83]:
abstract class MyLinearModel[FeaturesType, Model <: MyLinearModel[FeaturesType, Model]]
  extends PredictionModel[FeaturesType, Model] {
}

class MyLinearModelImpl(override val uid: String, val weights: Vector, val trainingError: Array[Double])
    extends MyLinearModel[Vector, MyLinearModelImpl] {

  override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)

  def predict(features: Vector): Double = {
    println("Predicting")
    val prediction = Helper.dot(weights, features)
    prediction
  }
}

Name: Compile Error
Message: <console>:16: error: not found: type PredictionModel
         extends PredictionModel[FeaturesType, Model] {
                 ^
<console>:20: error: type Vector takes type parameters
           extends MyLinearModel[Vector, MyLinearModelImpl] {
                                 ^
<console>:19: error: type Vector takes type parameters
       class MyLinearModelImpl(override val uid: String, val weights: Vector, val trainingError: Array[Double])
                                                                      ^
<console>:22: error: not found: type ParamMap
         override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)
                                  ^
<console>:22: error: not found: value defaultCopy
         override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)
                                                                 ^
<console>:24: error: type Vector takes type parameters
         def predict(features:

In [84]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearRegression[
    FeaturesType,
    Learner <: MyLinearRegression[FeaturesType, Learner, Model],
    Model <: MyLinearModel[FeaturesType, Model]]
  extends Predictor[FeaturesType, Learner, Model] {
}

class MyLinearRegressionImpl(override val uid: String)
    extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
  def this() = this(Identifiable.randomUID("linReg"))

  override def copy(extra: ParamMap): MyLinearRegressionImpl = defaultCopy(extra)
  
  def train(dataset: Dataset[_]): MyLinearModelImpl = {
    println("Training")

    val numIters = 10

    val instances: RDD[Instance] = dataset.select(
      col($(labelCol)), col($(featuresCol))).rdd.map {
        case Row(label: Double, features: Vector) =>
          Instance(label, features)
      }

    val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)

    new MyLinearModelImpl(uid, weights, trainingError)
  }
}

Name: Compile Error
Message: <console>:34: error: not found: type MyLinearModel
           Model <: MyLinearModel[FeaturesType, Model]]
                    ^
<console>:39: error: not found: type MyLinearModelImpl
           extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
                                                                      ^
<console>:44: error: not found: type MyLinearModelImpl
         def train(dataset: Dataset[_]): MyLinearModelImpl = {
                                         ^
<console>:55: error: not found: type LR
           val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)
                                              ^
<console>:57: error: not found: type MyLinearModelImpl
           new MyLinearModelImpl(uid, weights, trainingError)
               ^

StackTrace: 

In [85]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
val model = lr.fit(trainSet)
val predictions = model.transform(trainSet)
predictions.select("prediction", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

Name: Compile Error
Message: <console>:81: error: not found: type MyLinearRegressionImpl
       val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
                    ^

StackTrace: 

---
# 9. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

In [86]:
val data = spark.read.format("csv").option("header", "true").option("inferSchema","true")load("data/ccdefault.csv")

data = [ID: int, LIMIT_BAL: int ... 23 more fields]


[ID: int, LIMIT_BAL: int ... 23 more fields]

In [87]:
data.count()

30000

In [88]:
data.show(10)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|     3261|       0|    1000|    1000|    1000|       0|    2000|    

In [89]:
data.columns

Array(ID, LIMIT_BAL, SEX, EDUCATION, MARRIAGE, AGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6, DEFAULT)

In [90]:
data.describe().select("ID","LIMIT_BAL","SEX","AGE").show(5)

+-----------------+------------------+------------------+-----------------+
|               ID|         LIMIT_BAL|               SEX|              AGE|
+-----------------+------------------+------------------+-----------------+
|            30000|             30000|             30000|            30000|
|          15000.5|167484.32266666667|1.6037333333333332|          35.4855|
|8660.398374208891|129747.66156720246|0.4891291960902602|9.217904068090155|
|                1|             10000|                 1|               21|
|            30000|           1000000|                 2|               79|
+-----------------+------------------+------------------+-----------------+



In [91]:
val renamedData = data.withColumnRenamed("DEFAULT","label");

renamedData = [ID: int, LIMIT_BAL: int ... 23 more fields]


[ID: int, LIMIT_BAL: int ... 23 more fields]

In [92]:
renamedData.select("label").show(5)

+-----+
|label|
+-----+
|    1|
|    1|
|    0|
|    0|
|    0|
+-----+
only showing top 5 rows



In [93]:
val colLabel = "label"
val colNum = renamedData.columns.filter(_ != colLabel)

colLabel = label
colNum = Array(ID, LIMIT_BAL, SEX, EDUCATION, MARRIAGE, AGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)


Array(ID, LIMIT_BAL, SEX, EDUCATION, MARRIAGE, AGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)

In [94]:
renamedData.select("label").show(5)

+-----+
|label|
+-----+
|    1|
|    1|
|    0|
|    0|
|    0|
+-----+
only showing top 5 rows



In [95]:
for (c <- colNum) {
    println(c)
   val a =renamedData.where(renamedData.col(c).isNull).count()

   println(a) 
}

ID
0
LIMIT_BAL
0
SEX
0
EDUCATION
0
MARRIAGE
0
AGE
0
PAY_0
0
PAY_2
0
PAY_3
0
PAY_4
0
PAY_5
0
PAY_6
0
BILL_AMT1
0
BILL_AMT2
0
BILL_AMT3
0
BILL_AMT4
0
BILL_AMT5
0
BILL_AMT6
0
PAY_AMT1
0
PAY_AMT2
0
PAY_AMT3
0
PAY_AMT4
0
PAY_AMT5
0
PAY_AMT6
0


In [96]:
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("featuresFlat")
val featuredData = va.transform(renamedData)

val scaler = new StandardScaler().setInputCol("featuresFlat").setOutputCol("scalar")
val scaledData = scaler.fit(featuredData).transform(featuredData)

scaledData.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|        featuresFlat|              scalar|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|    1|[1.0,20000.0,2.0,...|[1.15468129385139...|
|  2|   120000|  2|        2|       

va = vecAssembler_7babbd324ff9
featuredData = [ID: int, LIMIT_BAL: int ... 24 more fields]
scaler = stdScal_025c35f9d471
scaledData = [ID: int, LIMIT_BAL: int ... 25 more fields]


[ID: int, LIMIT_BAL: int ... 25 more fields]

In [97]:
scaledData.columns

Array(ID, LIMIT_BAL, SEX, EDUCATION, MARRIAGE, AGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6, label, featuresFlat, scalar)

In [98]:
scaledData.select("label").distinct().count()

2

In [99]:
val cols=Array("scalar")
print(cols)

[Ljava.lang.String;@2a74af64

cols = Array(scalar)


Array(scalar)

In [100]:
// val newData = pipeline.fit(scaledData).transform(scaledData)

val va2 = new VectorAssembler().setInputCols(cols).setOutputCol("features")
val dataset = va2.transform(scaledData).select("features", "label")

va2 = vecAssembler_9f8914467752
dataset = [features: vector, label: int]


[features: vector, label: int]

In [101]:
dataset.select("label").show(5)

+-----+
|label|
+-----+
|    1|
|    1|
|    0|
|    0|
|    0|
+-----+
only showing top 5 rows



In [102]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(24,[0,1,2,3,4,5,...|    0|
|(24,[0,1,2,3,4,5,...|    1|
|(24,[0,1,2,3,4,5,...|    1|
|(24,[0,1,2,3,4,5,...|    0|
|(24,[0,1,2,3,4,5,...|    1|
+--------------------+-----+
only showing top 5 rows



trainSet = [features: vector, label: int]
testSet = [features: vector, label: int]


[features: vector, label: int]

In [103]:
import org.apache.spark.ml.classification.{BinaryLogisticRegressionSummary, LogisticRegression}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics 
import org.apache.spark.ml.evaluation.Evaluator

In [104]:
val lr = new LogisticRegression()
.setFeaturesCol("features").setLabelCol("label")
val lrModel = lr.fit(trainSet)

val trainingSummary = lrModel.summary
print(trainingSummary)

org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@10b5c66d

lr = logreg_a9f19a5364d1
lrModel = LogisticRegressionModel: uid = logreg_a9f19a5364d1, numClasses = 2, numFeatures = 24
trainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@10b5c66d


org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@10b5c66d

In [105]:
print(s"The accuracy of the classifier is = ${trainingSummary.accuracy}")

The accuracy of the classifier is = 0.8111296373488954

In [106]:
val predictions = lrModel.transform(testSet)
predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[1.04253590436076...|[0.73933901408677...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[2.68903390838870...|[0.93637645044229...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.68119143194593...|[0.66400456077321...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[0.89842532569589...|[0.71062579911165...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[1.11114456755230...|[0.75234243262797...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



predictions = [features: vector, label: int ... 3 more fields]


[features: vector, label: int ... 3 more fields]

In [107]:
val lp = predictions.select( "label", "prediction")
val counttotal = predictions.count()
val correct = lp.filter($"label" === $"prediction").count()
val wrong = lp.filter(not($"label" === $"prediction")).count()
val truep = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count()
val falseN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count()
val falseP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count()
val ratioWrong=wrong.toDouble/counttotal.toDouble
val ratioCorrect=correct.toDouble/counttotal.toDouble

lp = [label: int, prediction: double]
counttotal = 6010
correct = 4862
wrong = 1148
truep = 4543
falseN = 1024
falseP = 124
ratioWrong = 0.19101497504159734
ratioCorrect = 0.8089850249584026


0.8089850249584026

In [108]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val evaluator1 = new BinaryClassificationEvaluator().setMetricName("areaUnderROC")
// Evaluates predictions and returns a scalar metric areaUnderROC(larger is better).**
// val evaluator = new BinaryClassificationEvaluator()
val accuracy = evaluator1.evaluate(predictions)
print(accuracy)
// val evaluator = new RegressionEvaluator().setMetricName("areaUnderROC")
// .setPredictionCol("prediction")
// .setLabelCol("label")
// val rmse = evaluator.evaluate(predictions)
// println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

0.713963362791393

evaluator1 = binEval_8b12ce6ddc36
accuracy = 0.713963362791393


0.713963362791393

In [109]:
predictions.select("prediction", "label", "features").where("prediction == 0").show(15)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 15 rows



In [110]:
val objectiveHistory = trainingSummary.objectiveHistory
objectiveHistory.foreach(loss => println(loss))

0.5277087153606156
0.4850826070214496
0.47890273983989573
0.4780410996278998
0.4738331046448573
0.4701638026206252
0.4654859456937353
0.46496422720820413
0.46432237239861307
0.46419765574454136
0.46405057439930025
0.4638799323662472
0.46368313414222734
0.4635756467987573
0.4634731806815315
0.46342209435261367
0.4633860260403134
0.4633591786219933
0.4632957562544882
0.46323614518695233
0.4631026909698114
0.4630157756801869
0.46292632605654216
0.4628742487378277
0.46286914651775884
0.46284420616731997
0.4628365941640002
0.4628201970011964
0.46281242426848673
0.4628024683377443
0.46278783461423056
0.46277632726132284
0.4627563836540671
0.4627419415588407
0.46272946605785176
0.4627220085883122
0.4627185163114392
0.4627160547588746
0.46271328769203707
0.46270970014855567
0.46270914132204427
0.46270652534601137
0.46270389068772366
0.462702416691782
0.46270168810561824
0.4627010672322048
0.46270054086508516
0.46270036523778707
0.4627001286598216
0.46269997572460453
0.46269982314496577
0.46269

objectiveHistory = Array(0.5277087153606156, 0.4850826070214496, 0.47890273983989573, 0.4780410996278998, 0.4738331046448573, 0.4701638026206252, 0.4654859456937353, 0.46496422720820413, 0.46432237239861307, 0.46419765574454136, 0.46405057439930025, 0.4638799323662472, 0.46368313414222734, 0.4635756467987573, 0.4634731806815315, 0.46342209435261367, 0.4633860260403134, 0.4633591786219933, 0.4632957562544882, 0.46323614518695233, 0.4631026909698114, 0.4630157756801869, 0.46292632605654216, 0.4628742487378277, 0.46286914651775884, 0.46284420616731997, 0.4628365941640002, 0.4628201970011964, 0.46281242426848673, 0.4628024683377443, 0.46278783461423056, 0.46277632726132284, 0.4627563836540671, 0.4627419415588407, 0.46272946605785176, 0.4627220085883122, 0.4627185163114392, 0....


Array(0.5277087153606156, 0.4850826070214496, 0.47890273983989573, 0.4780410996278998, 0.4738331046448573, 0.4701638026206252, 0.4654859456937353, 0.46496422720820413, 0.46432237239861307, 0.46419765574454136, 0.46405057439930025, 0.4638799323662472, 0.46368313414222734, 0.4635756467987573, 0.4634731806815315, 0.46342209435261367, 0.4633860260403134, 0.4633591786219933, 0.4632957562544882, 0.46323614518695233, 0.4631026909698114, 0.4630157756801869, 0.46292632605654216, 0.4628742487378277, 0.46286914651775884, 0.46284420616731997, 0.4628365941640002, 0.4628201970011964, 0.46281242426848673, 0.4628024683377443, 0.46278783461423056, 0.46277632726132284, 0.4627563836540671, 0.4627419415588407, 0.46272946605785176, 0.4627220085883122, 0.4627185163114392, 0....

In [111]:
// Obtain the metrics useful to judge performance on test data.
// We cast the summary to a BinaryLogisticRegressionSummary since the problem is a
// binary classification problem.
val binarySummary = trainingSummary.asInstanceOf[BinaryLogisticRegressionSummary]

// Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
val roc = binarySummary.roc
roc.show()
println(binarySummary.areaUnderROC)

// Set the model threshold to maximize F-Measure
val fMeasure = binarySummary.fMeasureByThreshold
val maxFMeasure = fMeasure.select(max("F-Measure")).head().getDouble(0)
val bestThreshold = fMeasure.where($"F-Measure" === maxFMeasure)
  .select("threshold").head().getDouble(0)
lrModel.setThreshold(bestThreshold)

+--------------------+-------------------+
|                 FPR|                TPR|
+--------------------+-------------------+
|                 0.0|                0.0|
|0.003209070973953...|0.03381825051955413|
|0.006846018077766...| 0.0661250708482902|
|0.009841150986789324|0.10069903646325336|
|0.013638551639300422|0.13243907047043266|
|0.017222014226881317|0.16493481957302097|
|0.021179868428090068| 0.1961080672586435|
| 0.02519120714553137|0.22709238617041375|
|0.030379205220088783|0.25392027205743434|
|0.035781141359576406| 0.2799924428490459|
| 0.04059474782050596| 0.3081428301530323|
|0.046317591057388884| 0.3330814283015303|
| 0.05187998074557416| 0.3585868127715851|
| 0.05867251430710809|  0.379746835443038|
| 0.06605337754720009|  0.398828641602116|
| 0.07354120981975718|0.41753259021348954|
| 0.08204524790073274| 0.4326468921216701|
| 0.09060277049794084|0.44757226525599847|
| 0.09916029309514895|0.46249763839032687|
| 0.10809220730598491| 0.4761005101076894|
+----------

binarySummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@10b5c66d
roc = [FPR: double, TPR: double]
fMeasure = [threshold: double, F-Measure: double]
maxFMeasure = 0.5126575995234786
bestThreshold = 0.2868251324470622


LogisticRegressionModel: uid = logreg_a9f19a5364d1, numClasses = 2, numFeatures = 24

# Decision Tree

In [121]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.classification.DecisionTreeClassifier

val dt = new DecisionTreeClassifier().setLabelCol("label").setFeaturesCol("features")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
// val evaluator = new RegressionEvaluator().setMetricName("rmse")
// .setPredictionCol("prediction")
// .setLabelCol("label")
// val rmse = evaluator.evaluate(predictions)
// println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows



dt = dtc_0c299493da42
dtModel = DecisionTreeClassificationModel (uid=dtc_0c299493da42) of depth 5 with 35 nodes
predictions = [features: vector, label: int ... 3 more fields]


[features: vector, label: int ... 3 more fields]

In [120]:
val evaluator = new BinaryClassificationEvaluator()
val accuracy = evaluator.evaluate(predictions)

print(accuracy)

0.32063365009083755

evaluator = binEval_f24a02973630
accuracy = 0.32063365009083755


0.32063365009083755

In [113]:
val lp = predictions.select( "label", "prediction")
val counttotal = predictions.count()
val correct = lp.filter($"label" === $"prediction").count()
val wrong = lp.filter(not($"label" === $"prediction")).count()
val truep = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count()
val falseN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count()
val falseP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count()
val ratioWrong=wrong.toDouble/counttotal.toDouble
val ratioCorrect=correct.toDouble/counttotal.toDouble

lp = [label: int, prediction: double]
counttotal = 6010
correct = 4922
wrong = 1088
truep = 4453
falseN = 874
falseP = 214
ratioWrong = 0.1810316139767055
ratioCorrect = 0.8189683860232945


0.8189683860232945

# Random Forest

In [123]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.classification.RandomForestClassifier

val rf = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features").setNumTrees(10)

// train the model
val rfModel = rf.fit(trainSet)

//make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(15)

// select (prediction, true label) and compute test error
// val evaluator = new RegressionEvaluator().setMetricName("rmse")
// .setPredictionCol("prediction")
// .setLabelCol("label")
// val rmse = evaluator.evaluate(predictions)
// println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    0|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
|       0.0|    1|(24,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 15 rows



rf = rfc_88cf36f9f257
rfModel = RandomForestClassificationModel (uid=rfc_88cf36f9f257) with 10 trees
predictions = [features: vector, label: int ... 3 more fields]


[features: vector, label: int ... 3 more fields]

In [124]:
predictions.select("prediction", "label", "features").where("prediction == 1").show(15)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       1.0|    1|(24,[0,1,2,3,4,5,...|
|       1.0|    1|(24,[0,1,2,3,4,5,...|
|       1.0|    1|(24,[0,1,2,3,4,5,...|
|       1.0|    1|(24,[0,1,2,3,4,5,...|
|       1.0|    1|(24,[0,1,2,3,4,5,...|
|       1.0|    0|[0.00635074711618...|
|       1.0|    0|[0.00681261963372...|
|       1.0|    0|[0.01039213164466...|
|       1.0|    1|[0.01050759977404...|
|       1.0|    0|[0.01859036883100...|
|       1.0|    1|[0.02263175335948...|
|       1.0|    1|[0.03117639493398...|
|       1.0|    0|[0.03810448269709...|
|       1.0|    1|[0.04410882542512...|
|       1.0|    1|[0.04676459240098...|
+----------+-----+--------------------+
only showing top 15 rows



In [128]:
val evaluator = new BinaryClassificationEvaluator()
val accuracy = evaluator.evaluate(predictions)

print(accuracy)

evaluator.getMetricName()

Name: Compile Error
Message: <console>:97: error: not enough arguments for method apply: (index: Int)Char in class StringOps.
Unspecified value parameter index.
       evaluator.getMetricName()
                              ^

StackTrace: 

In [116]:
val lp = predictions.select( "label", "prediction")
val counttotal = predictions.count()
val correct = lp.filter($"label" === $"prediction").count()
val wrong = lp.filter(not($"label" === $"prediction")).count()
val truep = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count()
val falseN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count()
val falseP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count()
val ratioWrong=wrong.toDouble/counttotal.toDouble
val ratioCorrect=correct.toDouble/counttotal.toDouble

lp = [label: int, prediction: double]
counttotal = 6010
correct = 4896
wrong = 1114
truep = 4468
falseN = 915
falseP = 199
ratioWrong = 0.18535773710482528
ratioCorrect = 0.8146422628951747


0.8146422628951747